In [1]:
from data_transformers import chain
import pandas as pd
from analysislib import (
    get_geonomenclador, 
    exportar_definitivo,
    exportar_transformador,
    match_relocate,
)

from data_transformers.default_transformers import (
    drop_col,
    wide_to_long,
    replace_value,
    rename_cols,
    sort_values,
    cast_col
)

geonomenclador = get_geonomenclador()

def compare_ids(a):
    def _(b):
        try:
            a_before, a_after = a.split('_g')
            b_before, b_after = b.split('_g')
            return a_before == b_before and int(a_after) == int(b_after)
        except Exception:
            return False
    return _

In [2]:
from dotenv import load_dotenv
from analysislib import gsheet_download_xlsx
import os

ARGENDATA_WORK_ID = os.getenv("ARGENDATA_WORK_ID")
gsheet_download_xlsx(id= ARGENDATA_WORK_ID, target="work.xlsx")

work = pd.read_excel('./work.xlsx', sheet_name='Suma de gráficos')
tipo = work[['ID', 'Unnamed: 7', 'Bajada','Publicación']]

In [13]:
SUBTOP = 'PRECIO'
entrega = 1
folder = f'../output/{SUBTOP}{entrega}'
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n:02d}'
df = pd.read_csv(f'{folder}/definitivos/{archivo}.csv')

exportar = (lambda df:
                exportar_definitivo(archivo=archivo,
                                    folder=folder,
                                    df=df,
                                    nuevo_nombre=None))

_, tipologia_target, bajada, va = tipo[tipo.ID.map(compare_ids(archivo))].iloc[0]

va = va.lower() != 'no va'

print(f"Va? = {'Sí' if va else 'No'}")
print(f"Tipología target = '{tipologia_target}'")
print(f"Bajada = '{bajada}'")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print('Primary keys:', ', '.join(f'"{x}"' for x in primary_keys))
df.head()

Va? = Sí
Tipología target = 'Barras apiladas con selector'
Bajada = 'Composición del Índice de Precios al Consumidor en Argentina, por rubro y región (2016)'
Primary keys: "region", "sector"


,region,sector,valor
0,Nacional,Alimentos y bebidas no alcoholicas,26.9
1,GBA,Alimentos y bebidas no alcoholicas,23.4
2,Pampeana,Alimentos y bebidas no alcoholicas,28.6
3,NEA,Alimentos y bebidas no alcoholicas,35.3
4,NOA,Alimentos y bebidas no alcoholicas,34.7


In [20]:
df.region.unique()

array(['Nacional', 'GBA', 'Pampeana', 'NEA', 'NOA', 'Cuyo', 'Patagonia'],
      dtype=object)

In [4]:
pipeline = chain(
    rename_cols({'sector': 'indicador'})
)

callstack, result = pipeline(df)

result.head()

,indicador,valor
0,Alimentos y bebidas no alcoholicas,26.93381
1,Bebidas alcoholicas y tabaco,3.49540
2,Prendas de vestir y calzado,9.90447
3,Vivienda agua electricidad gas y otros combust...,9.43438
4,Equipamiento y mantenimiento del hogar,6.36303


In [5]:
exportar_transformador(archivo, pipeline, callstack)

'PRECIO_g01_transformer.py'

In [12]:
exportar(result)

('../output/PRECIO1/definitivos/PRECIO_g01.csv',
 '../output/PRECIO1/definitivos/PRECIO_g01_old.csv')